# Check your installation

The purpose of this tutorial is to make sure that you installed everything that is needed and the WannierBerri runs smoothly on your computer. If you run all the cells without errors, you are good to go (even if you do not understand what is going on)

# First, import needed modules

In [ ]:
# So, first import the necessary modules

import wannierberri as wberri
print (f"Using WannierBerri version {wberri.__version__}")
import pythtb
import numpy as np
import matplotlib.pyplot as plt

## check parallel installation

In [ ]:
#  This block is needed if you run this cell for a second time
#  because one cannot initiate two parallel environments at a time
try:
    parallel.shutdown()
except NameError:
    pass

# Chiose one of the options:

parallel = wberri.Parallel(num_cpus=2)
#parallel = wberri.Parallel()  # automatic detection
#parallel = wberri.Serial()

## Good. Now let's create a k.p model.

For simplicity, it is a 1-band free-electron

In [ ]:
# hamiltonian takes k-vector in reciprocal angstroms and gives hamiltonian in eV
mass = 1.5
def ham(k):
    return np.array([[np.dot(k,k)/(2*mass)]])
# k.p model is bounded to a box of size 2*kmax
system_kp = wberri.system.SystemKP(Ham=ham, kmax=2)

In [ ]:
path=wberri.Path(system_kp,
                 k_nodes=[
        [0.0000, 0.0000, 0.0000 ],   #  G
        [1.0 ,1.0, 1.0],   #  G
        [0.0, 1.0, 1.0],   #  G
        [0.0,1.0,0.0]   #  G
                 ] ,
                 #labels=["G0","G111","G011","G010"],
                 length=200 )   # length [ Ang] ~= 2*pi/dk

tabulators = { "Energy": wberri.calculators.tabulate.Energy(),
             }

tab_all_path = wberri.calculators.TabulatorAll(
                    tabulators,
                    # ibands = np.arange(0,18),
                    mode = "path"
                        )

In [ ]:
result=wberri.run(system_kp,
                  grid=path,
                  calculators = {"tabulate" : tab_all_path},
                  parallel = parallel,
                  print_Kpoints = False)

print (result.results)
path_result = result.results["tabulate"]

## Plot bands

The `TABresult` object already provides methods to plot the results. (As well as one can extract the data and plot them by other means). Below let’s plot the interpolated bands and compare with those obtained in QE. (file “bands/Fe_bands_pw.dat” is already provided)

In [ ]:
path_result = result.results["tabulate"]

plt.close()
path_result.plot_path_fat( path,
              quantity=None,
#              Eshift=EF,
              Emin=-2,  Emax=5,
              iband=None,
              mode="fatband",
              fatfactor=20,
              cut_k=False,
              close_fig=False,
              show_fig=False,
              label = "WB"
              )

kline = path.getKline()
plt.plot(kline,kline**2/(2*mass),"--",label="k**2/(2*mass)")

plt.legend()
plt.show()
plt.close()

In [ ]:
Efermi = np.linspace(-1,2,201)
calculators = {"dos":wberri.calculators.static.DOS(Efermi=Efermi,tetra=True)}
grid = wberri.Grid(system_kp, NK=20, NKFFT=10 )
result_run = wberri.run(system_kp,
            grid=grid,
            calculators = calculators,
            parallel=parallel,
            adpt_num_iter=0,
            fout_name='kp',
            restart=False,
            #print_Kpoints=False
            #file_Klist="Klist_ahc.pickle"  # needed to restart a calculation in future
            )

# Plot the DOS and compare with the analytical value

In [ ]:
dos = result_run.results["dos"].data
plt.plot(Efermi,dos)
plt.ylim(0,1)

Efpos=Efermi[Efermi>0.01]

dos_prec = np.sqrt(2*mass*Efpos)*system_kp.cell_volume*mass/(2*np.pi**2)

plt.plot(Efpos,dos_prec,"--")


# Now, some tight-binding models

In [ ]:
def Haldane_ptb(delta=0.2, hop1=-1.0, hop2=0.15, phi=np.pi / 2):
    """
    Defines a Haldane model within `TBmodels <https://tbmodels.greschd.ch>`__

    Parameters
    -----------
    delta : float
        difference between the on-site potentials of the two atoms
    t : float
        nearest-neighbour hopping
    hop2 : float
        magnitude of next nearest-neighbour hopping
    phi : float
        phase of next nearest-neighbour hopping

    Note:
    --------
    TBmodels  should be installed to use this (`pip install tbmodels`)
    """
    import pythtb
    lat = [[1.0, 0.0], [0.5, np.sqrt(3.0) / 2.0]]
    orb = [[1. / 3., 1. / 3.], [2. / 3., 2. / 3.]]

    my_model = pythtb.tb_model(2, 2, lat, orb)

    delta = 0.2
    t2 = hop2 * np.exp(1.j * phi)
    t2c = t2.conjugate()

    my_model.set_onsite([-delta, delta])
    my_model.set_hop(hop1, 0, 1, [0, 0])
    my_model.set_hop(hop1, 1, 0, [1, 0])
    my_model.set_hop(hop1, 1, 0, [0, 1])
    my_model.set_hop(t2, 0, 0, [1, 0])
    my_model.set_hop(t2, 1, 1, [1, -1])
    my_model.set_hop(t2, 1, 1, [0, 1])
    my_model.set_hop(t2c, 1, 1, [1, 0])
    my_model.set_hop(t2c, 0, 0, [1, -1])
    my_model.set_hop(t2c, 0, 0, [0, 1])

    return my_model

model = Haldane_ptb(delta=0.2, hop1=-1.0, hop2=0.15, phi=np.pi / 2)
system_haldane = wberri.system.System_PythTB(model,use_wcc_phase=True)


In [ ]:
path=wberri.Path(system_haldane,
                 k_nodes=[
        [0.0000, 0.0000, 0.0000 ],   #  G
        [1/3 ,1/3, 0],   #  K
        [0.0, 1/2, 0],   #  M
        [0.0,0.0,0.0]   #  G
                 ] ,
                 #labels=["G","K","M","G"],
                 length=200 )   # length [ Ang] ~= 2*pi/dk

tabulators = { "Energy": wberri.calculators.tabulate.Energy(),
             }

tab_all_path = wberri.calculators.TabulatorAll(
                    tabulators,
                    # ibands = np.arange(0,18),
                    mode = "path"
                        )

result=wberri.run(system_haldane,
                  grid=path,
                  calculators = {"tabulate" : tab_all_path},
                  parallel = parallel,
                  print_Kpoints = False)

path_result = result.results["tabulate"]

In [ ]:
path_result.plot_path_fat( path,
              quantity=None,
        #save_file="Fe_bands+QE.pdf",
#              Eshift=EF,
#              Emin=-2,  Emax=5,
              iband=None,
              mode="fatband",
              fatfactor=20,
              cut_k=False,
              close_fig=True,
              show_fig=True,
              label = "WB"
              )

In [ ]:
Efermi_haldane = np.linspace(-4,4,201)
calculators = {
               "dos":wberri.calculators.static.DOS(Efermi=Efermi_haldane,tetra=True),
               "ahc":wberri.calculators.static.AHC(Efermi=Efermi_haldane,tetra=True,kwargs_formula={"external_terms":False})
}
grid = wberri.Grid(system_kp, NK=[20,20,1], NKFFT=[10,10,1] )

In [ ]:
result_run = wberri.run(system_haldane,
            grid=grid,
            calculators = calculators,
            parallel=parallel,
            fout_name='Haldane',
            )

In [ ]:
plt.close()
dos = result_run.results["dos"].data
plt.plot(Efermi,dos)



In [ ]:
plt.close()
ahc = result_run.results["ahc"].data[:,2]
plt.plot(Efermi,ahc)

In [ ]:
# The 2D conductance in the gap is 
ahc[Efermi_haldane==0]*system_haldane.real_lattice[2,2]*1e-10

In [ ]:
# This is precisely half of conductance quantum
from scipy.constants import physical_constants as pc
[k for k in pc.keys() if "conduc" in k]
pc['conductance quantum']
pc['conductance quantum'][0]/2
